# Pipeline Unificado de Entrenamiento + Explainability (SHAP manual)
Este notebook replica el pipeline optimizado y añade un bloque de explainability tipo SHAP manual, usando la clase Explainability sin librerías externas.

## 1. Configuración de entorno y paths

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))  # Permite importar módulos desde el directorio superior
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
# Importar modelos OPTIMIZADOS
from classes.FeedForwardNetwork_opt import FeedForwardNetwork_opt
from classes.TabNet_opt import TabNetNetwork_opt
from classes.MixtureDensityNetworks_opt import MixtureDensityNetworks_opt
from classes.BayesianNN_opt import BayesianNN_opt
from classes.MonteCarloDropoutNetwork_opt import MonteCarloDropoutNetwork_opt
# Para TabNet externo
from pytorch_tabnet.tab_model import TabNetRegressor
import datetime
from classes.BaseTrainer_opt import BaseTrainer_opt
# Utilidad: obtiene último modelo para cada tipo
import re
from datetime import datetime
import plotly.graph_objects as go
import traceback


# Parámetros globales y de entrenamiento (modificables al inicio)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TARGETS = ['Mo_cumulative']
hidden_dim = 528
num_epochs = 500
lr = 0.001

dh = DataHandler()
dh.DEVICE = DEVICE
dh.set_targets(TARGETS)
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)
# Split primero (en arrays), luego conversión a tensores
X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train_np, dtype=torch.float32).to(DEVICE)
X_val = torch.tensor(X_val_np, dtype=torch.float32).to(DEVICE)
y_train = torch.tensor(y_train_np, dtype=torch.float32).to(DEVICE)
y_val = torch.tensor(y_val_np, dtype=torch.float32).to(DEVICE)

model_types = ['FeedForward', 'TabNet', 'MDN', 'BayesianNN', 'MonteCarloDropout']
resultados_modelos = {}
retrain_models = False  # Flags whether to retrain models or not

if retrain_models:
    for MODEL_TYPE in model_types:
        print(f'Entrenando y evaluando modelo: {MODEL_TYPE}')

        resultados = BaseTrainer_opt.run_pipeline(
            model_type=MODEL_TYPE,
            X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val,
            scales=scales, dh=dh,
            hidden_dim=hidden_dim,
            n_components=5,  # Solo relevante para MDN
            device=DEVICE,
            lr=lr,
            num_epochs=num_epochs,
            save_model=True,
            plot_results=False,
            dropout_prob=0.2
        )

        # Desempaquetar resultados según el modelo
        if MODEL_TYPE in ['FeedForward', 'TabNet']:
            y_train_mo, y_pred_train_mo, y_val_mo, y_pred_val_mo = resultados
            std_train = None
            std_val = None
        elif MODEL_TYPE == 'MDN':
            y_train_mo, y_pred_train_mo, y_val_mo, y_pred_val_mo, std_train, std_val = resultados
        elif MODEL_TYPE in ['BayesianNN', 'MonteCarloDropout']:
            y_train_mo, mean_train_mo, y_val_mo, mean_val_mo, std_train, std_val = resultados
            y_pred_train_mo = mean_train_mo
            y_pred_val_mo = mean_val_mo

        resultados_modelos[MODEL_TYPE] = {
            'y_train': y_train_mo,
            'y_pred_train': y_pred_train_mo,
            'y_val': y_val_mo,
            'y_pred_val': y_pred_val_mo,
            'std_train': std_train,
            'std_val': std_val
        }
else:
    print("No se van a reentrenar los modelos, cargando resultados previos...")

INFO:classes.DataHandler:Configuración de logging inicializada correctamente.
INFO:root:Conectado a data/Tronaduras_vs_Sismicidad.db
INFO:classes.DataHandler:Columnas objetivo actualizadas: ['Mo_cumulative']
INFO:classes.DataHandler:Inicio de la carga de datos.
INFO:classes.DataHandler:Tabla Tabla_Unificada cargada correctamente.
INFO:classes.DataHandler:Tabla Variables_Description cargada correctamente.
INFO:classes.DataHandler:Conexión cerrada.
INFO:classes.DataHandler:Datos cargados exitosamente.
INFO:classes.DataHandler:Inicio del preprocesamiento de datos.
INFO:classes.DataHandler:Variables de entrada procesadas: ['Cobertura Total', 'Cobertura Primario', 'Tronadura_Largo de Perforación (m)', 'Tronadura_N° Tiros', 'Tronadura_N° Tiros Real', 'Tronadura_Kg. de explosivos tronadura', 'Tronadura_Tipo Explosivo', 'Destressing_Se realizó', 'Destressing_N° Tiros', 'Destressing_Kg. de explosivos', 'Destressing_Tipo Explosivo', 'Geotecnicas_UCS (MPa)', 'Geotecnicas_Modulo de Young (GPa)', '

✅ Datos leídos desde 'Processed_Data.Tabla_Unificada' y convertidos a DataFrame.
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
🔌 Conexión cerrada manualmente.
No se van a reentrenar los modelos, cargando resultados previos...


In [2]:
# Utilidad: agrupamos SHAP manual en función

def run_explainability(model_class, model_path, X_train, y_train, input_vars, device, metric_fn, hidden_dim, n_components=None):
    """
    Ejecuta SHAP manual para el modelo dado.
    model_class: clase del modelo (debe tener .model y .load_state_dict o .load_model)
    model_path: ruta al .pt
    X_train, y_train: arrays (no tensores)
    input_vars: lista de nombres de variables
    device: cpu/cuda
    metric_fn: función métrica
    hidden_dim: dimensión oculta
    n_components: solo para MDN
    """
    if model_path is None:
        print(f'No se encontró modelo {model_class.__name__}.')
        return None
    # Inicializa el modelo con el input_dim correcto
    input_dim = X_train.shape[1]
    if n_components:
        model_obj = model_class(input_dim, hidden_dim, n_components=n_components, device=device)
    else:
        model_obj = model_class(input_dim, hidden_dim, device)
    # Usa el método load_model del BaseTrainer_opt (ahora todas lo tienen)
    # Nota: para TabNet deberías agregar un chequeo si la extensión es zip, pero si todo es pt o pyro, así funciona
    model_name = model_class.__name__.replace("Network_opt", "").replace("MixtureDensity", "MDN").replace("BayesianNN", "BayesianNN").replace("FeedForward", "FeedForward").replace("TabNet", "TabNet").replace("MonteCarloDropout", "MonteCarloDropout")
    if 'Bayesian' in model_name:
        model_name = 'BayesianNN'  # para que calce
    model_obj.load_model(model_path, model_name)
    # Asegúrate de poner en modo eval si existe
    if hasattr(model_obj, 'model') and hasattr(model_obj.model, 'eval'):
        model_obj.model.eval()
    expl = Explainability()
    expl.set_properties(device)
    importancias = expl.shap(
        X=X_train,
        y=y_train,
        input_vars=input_vars,
        model=model_obj.model,
        metric_fn=metric_fn,
        value='mean',
        show_info=True
    )
    return importancias



# Clase Explainability optimizada
class Explainability:
    def __init__(self):
        self.device = None
    def set_properties(self, device):
        self.device = device
    def shap(self, X, y, input_vars, model, metric_fn, value='mean', show_info=False, width=1200, height=300):
        importances = []
        n_vars = X.shape[1]
        # y_tensor solo se crea una vez
        y_tensor = torch.tensor(y, dtype=torch.float32).to(self.device)
        # with torch.no_grad() envuelve todo el ciclo
        with torch.no_grad():
            for i in range(n_vars):
                X_mod = X.copy()
                var_name = input_vars[i]
                # Determinar el valor de reemplazo
                if isinstance(value, str):
                    if value == 'mean':
                        val = X[:, i].mean()
                    elif value == 'median':
                        val = np.median(X[:, i])
                    elif value == 'zero':
                        val = 0.0
                    elif value == 'one':
                        val = 1.0
                    else:
                        raise ValueError("value debe ser un número o uno de: 'mean', 'median', 'zero', 'one'")
                else:
                    val = value
                X_mod[:, i] = val
                X_mod_tensor = torch.tensor(X_mod, dtype=torch.float32).to(self.device)
                y_pred_out = model(X_mod_tensor)
                if isinstance(y_pred_out, tuple):
                    y_pred = y_pred_out[0]
                else:
                    y_pred = y_pred_out
                y_pred = y_pred.cpu().numpy()
                metric = metric_fn(y, y_pred)
                if show_info:
                    print(f'🧪 {i+1}/{n_vars} Condicionando variable: {var_name}, métrica: {metric:.4f}')
                importances.append(metric)

        if show_info:
            fig = go.Figure()
            fig.add_trace(go.Bar(x=input_vars, y=importances, marker_color='indianred'))
            fig.update_layout(title="Importancia de variables (tipo SHAP manual)", xaxis_title="Variable", yaxis_title="Métrica (mayor = más importante)", template="plotly_white", width=width, height=height)
            fig.show()
            idx_max = int(np.argmax(importances))
            var_mas_importante = input_vars[idx_max]
            print(f'🏆 La variable más importante según este análisis es: {var_mas_importante} (posición {idx_max+1})')
        
        return importances



def get_latest_model_file(nets_dir, model_name):
    pattern = re.compile(rf'{model_name}.*_(\d{{8}}_\d{{6}})\.pt$')
    latest_file = None
    latest_dt = None
    for fname in os.listdir(nets_dir):
        if fname.endswith('.pt'):
            match = pattern.search(fname)
            if match:
                dt_str = match.group(1)
                dt = datetime.strptime(dt_str, '%Y%m%d_%H%M%S')
                if (latest_dt is None) or (dt > latest_dt):
                    latest_dt = dt
                    latest_file = fname
    if latest_file:
        return os.path.join(nets_dir, latest_file)
    else:
        return None

nets_dir = 'nets'
latest_files = {}
for model in model_types:
    file_path = get_latest_model_file(nets_dir, model)
    if file_path:
        print(f"Último modelo para {model}: {file_path}")
        latest_files[model] = file_path
    else:
        print(f"No se encontró modelo para {model}")

# ---
# Variables de entrada aseguradas en orden correcto
input_vars = inputs.columns.tolist()

# Métrica para explainability
# (puedes reemplazar por cualquier métrica personalizada)
def metric_fn(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))


Último modelo para FeedForward: nets\modelo_FeedForward_20250603_163338.pt
No se encontró modelo para TabNet
Último modelo para MDN: nets\modelo_MDN_20250603_163353.pt
Último modelo para BayesianNN: nets\modelo_BayesianNN_20250603_163428.pt
Último modelo para MonteCarloDropout: nets\modelo_MonteCarloDropout_20250603_163430.pt


In [3]:
# Lista de modelos y los kwargs específicos de cada uno
modelos_explain = [
    # (ClaseModelo, nombre en latest_files, kwargs extra)
    (FeedForwardNetwork_opt, 'FeedForward', {}),
    (TabNetNetwork_opt, 'TabNet', {}),
    (MixtureDensityNetworks_opt, 'MDN', {'n_components': 5}),
    (BayesianNN_opt, 'BayesianNN', {}),
    (MonteCarloDropoutNetwork_opt, 'MonteCarloDropout', {})
]

for clase_modelo, nombre_archivo, kwargs in modelos_explain:
    print(f"⏩ SHAP manual para {nombre_archivo}:")
    try:
        run_explainability(
            clase_modelo,
            latest_files.get(nombre_archivo),
            X_train_np,
            y_train_np,
            input_vars,
            DEVICE,
            metric_fn,
            hidden_dim,
            **kwargs  # Esto agrega n_components si corresponde, vacío si no
        )
    except Exception as e:
        print(f"Error al ejecutar SHAP manual para {nombre_archivo}: {e}")
        traceback.print_exc()  # Esto imprime el stacktrace completo

        continue




⏩ SHAP manual para FeedForward:
🧪 1/53 Condicionando variable: Cobertura Total, métrica: 0.0025
🧪 2/53 Condicionando variable: Cobertura Primario, métrica: 0.0037
🧪 3/53 Condicionando variable: Tronadura_Largo de Perforación (m), métrica: 0.0216
🧪 4/53 Condicionando variable: Tronadura_N° Tiros, métrica: 0.0164
🧪 5/53 Condicionando variable: Tronadura_N° Tiros Real, métrica: 0.0142
🧪 6/53 Condicionando variable: Tronadura_Kg. de explosivos tronadura, métrica: 0.0047
🧪 7/53 Condicionando variable: Tronadura_Tipo Explosivo, métrica: 0.0019
🧪 8/53 Condicionando variable: Destressing_Se realizó, métrica: 0.0122
🧪 9/53 Condicionando variable: Destressing_N° Tiros, métrica: 0.0265
🧪 10/53 Condicionando variable: Destressing_Kg. de explosivos, métrica: 0.0200
🧪 11/53 Condicionando variable: Destressing_Tipo Explosivo, métrica: 0.0019
🧪 12/53 Condicionando variable: Geotecnicas_UCS (MPa), métrica: 0.0208
🧪 13/53 Condicionando variable: Geotecnicas_Modulo de Young (GPa), métrica: 0.0279
🧪 14/53

🏆 La variable más importante según este análisis es: Avance_Tipo de Explosivo (posición 30)
⏩ SHAP manual para TabNet:
No se encontró modelo TabNetNetwork_opt.
⏩ SHAP manual para MDN:
🧪 1/53 Condicionando variable: Cobertura Total, métrica: 0.3965
🧪 2/53 Condicionando variable: Cobertura Primario, métrica: 0.3966
🧪 3/53 Condicionando variable: Tronadura_Largo de Perforación (m), métrica: 0.3950
🧪 4/53 Condicionando variable: Tronadura_N° Tiros, métrica: 0.3958
🧪 5/53 Condicionando variable: Tronadura_N° Tiros Real, métrica: 0.3960
🧪 6/53 Condicionando variable: Tronadura_Kg. de explosivos tronadura, métrica: 0.3967
🧪 7/53 Condicionando variable: Tronadura_Tipo Explosivo, métrica: 0.3966
🧪 8/53 Condicionando variable: Destressing_Se realizó, métrica: 0.3961
🧪 9/53 Condicionando variable: Destressing_N° Tiros, métrica: 0.3975
🧪 10/53 Condicionando variable: Destressing_Kg. de explosivos, métrica: 0.3960
🧪 11/53 Condicionando variable: Destressing_Tipo Explosivo, métrica: 0.3966
🧪 12/53 C

🏆 La variable más importante según este análisis es: Estructura Crítica (posición 19)
⏩ SHAP manual para BayesianNN:
Error al ejecutar SHAP manual para BayesianNN: 'NoneType' object has no attribute 'cpu'
⏩ SHAP manual para MonteCarloDropout:
🧪 1/53 Condicionando variable: Cobertura Total, métrica: 0.0179
🧪 2/53 Condicionando variable: Cobertura Primario, métrica: 0.0186
🧪 3/53 Condicionando variable: Tronadura_Largo de Perforación (m), métrica: 0.0224
🧪 4/53 Condicionando variable: Tronadura_N° Tiros, métrica: 0.0231
🧪 5/53 Condicionando variable: Tronadura_N° Tiros Real, métrica: 0.0217
🧪 6/53 Condicionando variable: Tronadura_Kg. de explosivos tronadura, métrica: 0.0186
🧪 7/53 Condicionando variable: Tronadura_Tipo Explosivo, métrica: 0.0176
🧪 8/53 Condicionando variable: Destressing_Se realizó, métrica: 0.0229
🧪 9/53 Condicionando variable: Destressing_N° Tiros, métrica: 0.0308
🧪 10/53 Condicionando variable: Destressing_Kg. de explosivos, métrica: 0.0235
🧪 11/53 Condicionando vari

Traceback (most recent call last):
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_22108\2056070852.py", line 14, in <module>
    run_explainability(
    ~~~~~~~~~~~~~~~~~~^
        clase_modelo,
        ^^^^^^^^^^^^^
    ...<7 lines>...
        **kwargs  # Esto agrega n_components si corresponde, vacío si no
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_22108\1874465435.py", line 35, in run_explainability
    importancias = expl.shap(
        X=X_train,
    ...<5 lines>...
        show_info=True
    )
  File "C:\Users\Michael\AppData\Local\Temp\ipykernel_22108\1874465435.py", line 85, in shap
    y_pred = y_pred.cpu().numpy()
             ^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'cpu'


🏆 La variable más importante según este análisis es: Coordenadas_Cota (m) (posición 45)
